_한국어로 기계번역됨_


# 그래프에 교차 스레드 지속성을 추가하는 방법

<div class="admonition tip">
    <p class="admonition-title">전제 조건</p>
    <p>
        이 가이드는 다음에 대한 이해가 있을 것을 가정합니다:
        <ul>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/persistence/">
                    지속성
                </a>
            </li>
            <li>
                <a href="https://langchain-ai.github.io/langgraph/concepts/memory/">
                    메모리
                </a>
            </li>
            <li>
                <a href="https://python.langchain.com/docs/concepts/#chat-models/">
                    챗 모델
                </a>
            </li>             
        </ul>
    </p>
</div>

[이전 가이드](https://langchain-ai.github.io/langgraph/how-tos/persistence/)에서는 단일 [스레드]()에서 여러 상호작용 간에 그래프 상태를 지속하는 방법을 배웠습니다. LangGraph는 **여러 스레드** 간에 데이터를 지속할 수 있도록 합니다. 예를 들어, 사용자의 이름이나 선호도와 같은 정보를 공유 메모리에 저장하고 새로운 대화 스레드에서 재사용할 수 있습니다.

이 가이드에서는 [Store](https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore) 인터페이스를 사용하여 구현된 공유 메모리를 가진 그래프를 구축하고 사용하는 방법을 보여줍니다.

<div class="admonition note">
    <p class="admonition-title">참고</p>
    <p>
    이 가이드에서 사용되는 <code><a href="https://langchain-ai.github.io/langgraph/reference/store/#langgraph.store.base.BaseStore">Store</a></code> API에 대한 지원은 LangGraph <code>v0.2.32</code>에서 추가되었습니다.
    </p>
    <p>
    이 가이드에서 사용되는 <b>index</b> 및 <b>query</b> 인수에 대한 지원은 LangGraph <code>v0.2.54</code>에서 추가되었습니다.
    </p>
</div>

## 설정

먼저 필요한 패키지를 설치하고 API 키를 설정해 보겠습니다.


In [1]:
%%capture --no-stderr
%pip install -U langchain_openai langgraph


In [ ]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")
_set_env("OPENAI_API_KEY")


!!! 팁 "LangGraph 개발을 위해 [LangSmith](https://smith.langchain.com) 설정하기"

    LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 발견하고 성능을 향상시킬 수 있습니다. LangSmith를 사용하면 LangGraph로 구축된 LLM 앱을 디버그, 테스트 및 모니터링하는 데 도움이 되는 추적 데이터를 사용할 수 있습니다 — 시작하는 방법에 대한 자세한 내용은 [여기](https://docs.smith.langchain.com)를 참조하세요.


## 저장소 정의

이 예제에서는 사용자의 선호도에 대한 정보를 검색할 수 있는 그래프를 생성합니다. 이를 위해 `InMemoryStore`를 정의할 것입니다. 이는 메모리 내에 데이터를 저장하고 그 데이터를 쿼리할 수 있는 객체입니다. 그래프를 컴파일할 때 저장소 객체를 전달할 것입니다. 이렇게 하면 그래프의 각 노드가 저장소에 접근할 수 있게 되며, 노드 함수를 정의할 때 `store` 키워드 인자를 정의할 수 있고, LangGraph는 그래프와 함께 컴파일한 저장소 객체를 자동으로 전달합니다.

`Store` 인터페이스를 사용하여 객체를 저장할 때 두 가지를 정의합니다:

* 객체의 네임스페이스, 즉 튜플(디렉토리와 유사함)
* 객체 키(파일명과 유사함)

우리의 예제에서는 `("memories", <user_id>)`를 네임스페이스로 사용하고 각 새로운 기억을 위한 랜덤 UUID를 키로 사용할 것입니다.

중요하게도, 사용자를 결정하기 위해 `user_id`를 노드 함수의 config 키워드 인자를 통해 전달할 것입니다.

먼저, 일부 사용자에 대한 기억으로 이미 채워진 `InMemoryStore`를 정의해 보겠습니다.


In [3]:
from langgraph.store.memory import InMemoryStore
from langchain_openai import OpenAIEmbeddings

in_memory_store = InMemoryStore(
    index={
        "embed": OpenAIEmbeddings(model="text-embedding-3-small"),
        "dims": 1536,
    }
)


## 그래프 생성


In [4]:
import uuid
from typing import Annotated
from typing_extensions import TypedDict

from langchain_anthropic import ChatAnthropic
from langchain_core.runnables import RunnableConfig
from langgraph.graph import StateGraph, MessagesState, START
from langgraph.checkpoint.memory import MemorySaver
from langgraph.store.base import BaseStore


model = ChatAnthropic(model="claude-3-5-sonnet-20240620")


# NOTE: we're passing the Store param to the node --
# this is the Store we compile the graph with
def call_model(state: MessagesState, config: RunnableConfig, *, store: BaseStore):
    user_id = config["configurable"]["user_id"]
    namespace = ("memories", user_id)
    memories = store.search(namespace, query=str(state["messages"][-1].content))
    info = "\n".join([d.value["data"] for d in memories])
    system_msg = f"You are a helpful assistant talking to the user. User info: {info}"

    # Store new memories if the user asks the model to remember
    last_message = state["messages"][-1]
    if "remember" in last_message.content.lower():
        memory = "User name is Bob"
        store.put(namespace, str(uuid.uuid4()), {"data": memory})

    response = model.invoke(
        [{"role": "system", "content": system_msg}] + state["messages"]
    )
    return {"messages": response}


builder = StateGraph(MessagesState)
builder.add_node("call_model", call_model)
builder.add_edge(START, "call_model")

# NOTE: we're passing the store object here when compiling the graph
graph = builder.compile(checkpointer=MemorySaver(), store=in_memory_store)
# If you're using LangGraph Cloud or LangGraph Studio, you don't need to pass the store or checkpointer when compiling the graph, since it's done automatically.


<div class="admonition tip">
    <p class="admonition-title">참고</p>
    <p>
        LangGraph Cloud나 LangGraph Studio를 사용하고 있다면, 그래프를 컴파일할 때 store를 전달할 필요가 <strong>없습니다</strong>. 이는 자동으로 처리되기 때문입니다.
    </p>
</div>


그래프를 실행하라!


이제 구성에서 사용자 ID를 지정하고 모델에게 우리의 이름을 알려줍시다:


In [5]:
config = {"configurable": {"thread_id": "1", "user_id": "1"}}
input_message = {"role": "user", "content": "Hi! Remember: my name is Bob"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

Hi! Remember: my name is Bob
================================== Ai Message ==================================

Hello Bob! It's nice to meet you. I'll remember that your name is Bob. How can I assist you today?


In [6]:
config = {"configurable": {"thread_id": "2", "user_id": "1"}}
input_message = {"role": "user", "content": "what is my name?"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

what is my name?
================================== Ai Message ==================================

Your name is Bob.


이제 우리의 인메모리 스토어를 검사하고 실제로 사용자에 대한 메모리를 저장했는지 확인할 수 있습니다.


In [7]:
for memory in in_memory_store.search(("memories", "1")):
    print(memory.value)


{'data': 'User name is Bob'}


이제 그래프를 다른 사용자에 대해 실행하여 첫 번째 사용자에 대한 기억이 독립적인지 확인해 보겠습니다.


In [8]:
config = {"configurable": {"thread_id": "3", "user_id": "2"}}
input_message = {"role": "user", "content": "what is my name?"}
for chunk in graph.stream({"messages": [input_message]}, config, stream_mode="values"):
    chunk["messages"][-1].pretty_print()


================================ Human Message =================================

what is my name?
================================== Ai Message ==================================

I apologize, but I don't have any information about your name. As an AI assistant, I don't have access to personal information about users unless it has been specifically shared in our conversation. If you'd like, you can tell me your name and I'll be happy to use it in our discussion.
